In [1]:
import numpy as np, pandas as pd, xarray as xr
from pathlib import Path
import datetime, networkx as nx, yaml, warnings, sys, logging, shutil, papermill
from disjoint_set import DisjointSet
from helper import singleglob, nxrender, Step, dict_merge, PdfWriter, DictWriter, TableWriter, TextWriter
import matplotlib.pyplot as plt
import matplotlib as mpl, seaborn as sns
from tqdm.notebook import tqdm

In [2]:
base_folder =  Path("/media/filer2/T4/Clara/Data/Luisa/")
dest_base_folder = Path("/home/julienb/Documents/Data/Luisa/")
force=True

In [3]:

in_sessions = [f.parent for f in base_folder.glob("**/metadata.yaml")]
sessions = pd.DataFrame()
sessions["in_session"] = in_sessions
sessions["dest_folder"] = [dest_base_folder / s.relative_to(base_folder).with_name(f"Session--{s.name}") for s in in_sessions]
sessions["already_done"] = sessions["dest_folder"].apply(lambda d: (d/"analysis").exists())
sessions


,in_session,dest_folder,already_done
0,/media/filer2/T4/Clara/Data/Luisa/Rat101/opto_...,/home/julienb/Documents/Data/Luisa/Rat101/opto...,False
1,/media/filer2/T4/Clara/Data/Luisa/Rat101/opto_...,/home/julienb/Documents/Data/Luisa/Rat101/opto...,True
2,/media/filer2/T4/Clara/Data/Luisa/Rat101/opto_...,/home/julienb/Documents/Data/Luisa/Rat101/opto...,True
3,/media/filer2/T4/Clara/Data/Luisa/Rat101/opto_...,/home/julienb/Documents/Data/Luisa/Rat101/opto...,True
4,/media/filer2/T4/Clara/Data/Luisa/Rat101/opto_...,/home/julienb/Documents/Data/Luisa/Rat101/opto...,True
...,...,...,...
136,/media/filer2/T4/Clara/Data/Luisa/Rat246/opto_...,/home/julienb/Documents/Data/Luisa/Rat246/opto...,True
137,/media/filer2/T4/Clara/Data/Luisa/Rat246/opto_...,/home/julienb/Documents/Data/Luisa/Rat246/opto...,True
138,/media/filer2/T4/Clara/Data/Luisa/Rat246/opto_...,/home/julienb/Documents/Data/Luisa/Rat246/opto...,True
139,/media/filer2/T4/Clara/Data/Luisa/Rat246/opto_...,/home/julienb/Documents/Data/Luisa/Rat246/opto...,True


In [4]:
import contextlib
import os


df = sessions[["in_session", "dest_folder"]] if force else sessions.loc[~sessions["already_done"], ["in_session", "dest_folder"]]
progress = tqdm(df.iterrows(), total=len(df.index))
errors = []
for _, (s, dest_folder) in progress:
    progress.set_postfix_str(f"Copying {s.stem}, n_copy_err {len(errors)}")
    try:
        papermill.execute_notebook("copy_session_data.ipynb", "copy_session_data_tmp.ipynb", parameters=
                                dict(dest_folder = str(dest_folder), session_folder=str(s)), progress_bar=False)
    except Exception as e:
        e.add_note(f"Error for preparing session {s}")
        errors.append((s, e))
if len(errors) > 0:
    errors_df = pd.DataFrame(errors, columns=["session", "exception"])
    errors_df["err_message"] = errors_df["exception"].astype(str)
    display(errors_df[["session"]].to_string())
    raise ExceptionGroup("Errors during copy", [e for s, e in errors])
else:
    if Path("copy_session_data_tmp.ipynb").exists():
        Path("copy_session_data_tmp.ipynb").unlink()


  0%|          | 0/12 [00:00<?, ?it/s]

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0

Sessions with copy errors:[PosixPath('/media/filer2/T4/Clara/Data/Luisa/Rat101/opto_ALLTRIALS/Rat101_20240911'), PosixPath('/media/filer2/T4/Clara/Data/Luisa/Rat101/opto_Fastest/Rat101_20240909'), PosixPath('/media/filer2/T4/Clara/Data/Luisa/Rat101/opto_Fastest/Rat101_20240910'), PosixPath('/media/filer2/T4/Clara/Data/Luisa/Rat101/opto_MT/Rat101_20240731'), PosixPath('/media/filer2/T4/Clara/Data/Luisa/Rat226/opto_MT/Rat226_20240806'), PosixPath('/media/filer2/T4/Clara/Data/Luisa/Rat230/opto_MT/Rat230_20240801'), PosixPath('/media/filer2/T4/Clara/Data/Luisa/Rat233/opto_MT/Rat233_20240729'), PosixPath('/media/filer2/T4/Clara/Data/Luisa/Rat233/opto_MT/Rat233_20240731'), PosixPath('/media/filer2/T4/Clara/Data/Luisa/Rat233/opto_MT/Rat233_20240807'), PosixPath('/media/filer2/T4/Clara/Data/Luisa/Rat239/opto_MT/bilat/Rat239_20240729'), PosixPath('/media/filer2/T4/Clara/Data/Luisa/Rat240/opto_MT/Rat240_20240730'), PosixPath('/media/filer2/T4/Clara/Data/Luisa/Rat246/opto_MT/Rat246_20240731')]


  + Exception Group Traceback (most recent call last):
  |   File "/home/julienb/miniconda3/envs/dev/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
  |     exec(code_obj, self.user_global_ns, self.user_ns)
  |   File "/tmp/ipykernel_292368/3758791682.py", line 35, in <module>
  |     raise ExceptionGroup("Errors during copy", [e for s, e in copy_errors])
  | ExceptionGroup: Errors during copy (12 sub-exceptions)
  +-+---------------- 1 ----------------
    | Traceback (most recent call last):
    |   File "/tmp/ipykernel_292368/3758791682.py", line 15, in <module>
    |     papermill.execute_notebook("copy_session.ipynb", "copy_session_tmp.ipynb", parameters=
    |   File "/home/julienb/miniconda3/envs/dev/lib/python3.11/site-packages/papermill/execute.py", line 131, in execute_notebook
    |     raise_for_execution_errors(nb, output_path)
    |   File "/home/julienb/miniconda3/envs/dev/lib/python3.11/site-packages/papermill/execute.py", line 251